# Exercices : Data Engineering avec Python

🔴 **Avance** | ⏱ **2h** | 📚 **Sections validees : 06-Data-Engineering**

## Objectifs
- Manipuler des donnees avec Pandas
- Valider des donnees avec Pydantic
- Creer des pipelines de transformation
- Utiliser DuckDB pour des requetes SQL
- Exporter en format Parquet

## Projet : Analyse de Donnees E-commerce
Vous allez travailler avec un dataset de commandes e-commerce et creer un pipeline complet de traitement de donnees.

## Sections couvertes
1. Exploration avec Pandas (1 exercice)
2. Agregations et transformations (1 exercice)
3. Validation avec Pydantic (1 exercice)
4. Pipeline complet (1 exercice)
5. Analyse avec DuckDB (1 exercice)

---
## Preparation : Creation du Dataset

Creez le dataset de demonstration.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Seed pour la reproductibilite
np.random.seed(42)

# Generer 100 commandes sur 100 jours
data = {
    "order_id": range(1, 101),
    "date": [datetime(2024, 1, 1) + timedelta(days=i) for i in range(100)],
    "product": ["Laptop", "Phone", "Tablet", "Headphones", "Monitor"] * 20,
    "category": ["Electronics", "Electronics", "Electronics", "Audio", "Electronics"] * 20,
    "quantity": [1, 2, 1, 3, 1, 2, 1, 4, 1, 2] * 10,
    "unit_price": [999.99, 699.99, 449.99, 79.99, 349.99] * 20,
    "customer_id": [f"C{i:03d}" for i in range(1, 101)],
    "city": ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux"] * 20,
}

df = pd.DataFrame(data)
df["total"] = df["quantity"] * df["unit_price"]

# Ajouter quelques valeurs manquantes et aberrantes pour le realisme
df.loc[5, 'city'] = None
df.loc[10, 'quantity'] = -1  # Valeur aberrante
df.loc[15, 'unit_price'] = 0  # Prix invalide

print(f"Dataset cree : {len(df)} commandes")
print(f"Periode : {df['date'].min()} a {df['date'].max()}")
print(f"\nApercu :")
df.head(10)

---
## Exercice 1 : Exploration avec Pandas (FACILE)

Explorez le dataset et repondez aux questions suivantes :

1. Combien de commandes au total ?
2. Quelles sont les colonnes et leurs types ?
3. Y a-t-il des valeurs manquantes ?
4. Statistiques descriptives sur les colonnes numeriques
5. Filtrer les commandes avec un total > 1000 EUR
6. Trier par date decroissante

Utilisez : `info()`, `describe()`, `isnull()`, `sum()`, `query()`, `sort_values()`

In [ ]:
# Votre code ici


### Solution 1

In [ ]:
# Solution
print("=== EXPLORATION DU DATASET ===")

# 1. Nombre de commandes
print(f"\n1. Nombre de commandes : {len(df)}")

# 2. Colonnes et types
print("\n2. Informations sur les colonnes :")
print(df.info())

# 3. Valeurs manquantes
print("\n3. Valeurs manquantes :")
valeurs_manquantes = df.isnull().sum()
print(valeurs_manquantes[valeurs_manquantes > 0])

# 4. Statistiques descriptives
print("\n4. Statistiques descriptives :")
print(df.describe())

# Statistiques supplementaires
print("\nChiffre d'affaires total :", f"{df['total'].sum():,.2f} EUR")
print("Panier moyen :", f"{df['total'].mean():.2f} EUR")
print("Panier median :", f"{df['total'].median():.2f} EUR")

# 5. Filtrer les commandes > 1000 EUR
print("\n5. Commandes > 1000 EUR :")
commandes_importantes = df[df['total'] > 1000]
print(f"Nombre : {len(commandes_importantes)}")
print(commandes_importantes[['order_id', 'product', 'quantity', 'total']].head())

# Alternative avec query
commandes_importantes_v2 = df.query('total > 1000')
print(f"\nVerification avec query() : {len(commandes_importantes_v2)} commandes")

# 6. Trier par date decroissante
print("\n6. Dernieres commandes (triees par date) :")
df_trie = df.sort_values('date', ascending=False)
print(df_trie[['order_id', 'date', 'product', 'total']].head())

# Analyses supplementaires
print("\n=== ANALYSES SUPPLEMENTAIRES ===")

# Repartition par produit
print("\nNombre de commandes par produit :")
print(df['product'].value_counts())

# Repartition par ville
print("\nNombre de commandes par ville :")
print(df['city'].value_counts())

# Top 5 des plus grosses commandes
print("\nTop 5 des plus grosses commandes :")
print(df.nlargest(5, 'total')[['order_id', 'product', 'quantity', 'total']])

---
## Exercice 2 : Agregations et Transformations (MOYEN)

Creez les analyses suivantes avec `groupby()` et agregations :

1. **CA par produit** : Chiffre d'affaires total par produit
2. **CA par ville** : Chiffre d'affaires total par ville
3. **CA par mois** : Extraire le mois de la date et calculer le CA mensuel
4. **Quantites moyennes** : Quantite moyenne commandee par produit
5. **Analyse croisee** : CA par produit et par ville (pivot table)
6. **Top 3 produits** : Les 3 produits les plus vendus par ville

Bonus : Visualisez avec matplotlib ou plotly.

In [ ]:
# Votre code ici


### Solution 2

In [ ]:
# Solution
print("=== AGREGATIONS ET TRANSFORMATIONS ===")

# 1. CA par produit
print("\n1. CA par produit :")
ca_par_produit = df.groupby('product')['total'].sum().sort_values(ascending=False)
print(ca_par_produit)
print(f"\nProduit le plus vendu : {ca_par_produit.index[0]} ({ca_par_produit.iloc[0]:,.2f} EUR)")

# 2. CA par ville
print("\n2. CA par ville :")
ca_par_ville = df.groupby('city')['total'].sum().sort_values(ascending=False)
print(ca_par_ville)

# 3. CA par mois
print("\n3. CA par mois :")
df['mois'] = df['date'].dt.to_period('M')
ca_par_mois = df.groupby('mois')['total'].sum()
print(ca_par_mois)

# Calculer la croissance mensuelle
print("\nCroissance mensuelle :")
croissance = ca_par_mois.pct_change() * 100
print(croissance)

# 4. Quantites moyennes par produit
print("\n4. Quantites moyennes par produit :")
qty_moyenne = df.groupby('product')['quantity'].agg(['mean', 'sum', 'count'])
qty_moyenne.columns = ['Quantite_moyenne', 'Quantite_totale', 'Nb_commandes']
print(qty_moyenne)

# 5. Analyse croisee : CA par produit et ville
print("\n5. CA par produit et ville (pivot table) :")
pivot = df.pivot_table(
    values='total',
    index='product',
    columns='city',
    aggfunc='sum',
    fill_value=0
)
print(pivot)
print("\nTotal par ligne (produit) :")
pivot['TOTAL'] = pivot.sum(axis=1)
print(pivot.sort_values('TOTAL', ascending=False))

# 6. Top 3 produits par ville
print("\n6. Top 3 produits par ville :")
top3_par_ville = (
    df.groupby(['city', 'product'])['total']
    .sum()
    .sort_values(ascending=False)
    .groupby(level=0)
    .head(3)
)
print(top3_par_ville)

# Analyses supplementaires
print("\n=== ANALYSES SUPPLEMENTAIRES ===")

# Statistiques par categorie
print("\nStatistiques par categorie :")
stats_categorie = df.groupby('category').agg({
    'total': ['sum', 'mean', 'count'],
    'quantity': 'sum'
})
print(stats_categorie)

# Distribution des paniers
print("\nDistribution des montants de commandes :")
print(df['total'].describe(percentiles=[.25, .5, .75, .9, .95, .99]))

---
## Exercice 3 : Validation avec Pydantic (MOYEN)

Creez un modele Pydantic `Order` pour valider les donnees de commandes :

**Modele Order :**
- `order_id` : int > 0
- `date` : datetime
- `product` : str non vide
- `category` : str (doit etre dans ['Electronics', 'Audio', 'Accessories'])
- `quantity` : int > 0
- `unit_price` : float > 0
- `customer_id` : str (format "Cxxx")
- `city` : str non vide
- `total` : float (calcule automatiquement)

**Validateurs :**
- Valider le format du customer_id
- Calculer automatiquement le total
- Verifier que la date n'est pas dans le futur

Testez avec des donnees valides et invalides.

In [ ]:
# Votre code ici


### Solution 3

In [ ]:
# Solution
from pydantic import BaseModel, Field, field_validator, model_validator
from datetime import datetime
from typing import Literal
import re

class Order(BaseModel):
    """Modele Pydantic pour une commande e-commerce"""
    
    order_id: int = Field(gt=0, description="ID de la commande")
    date: datetime
    product: str = Field(min_length=1, description="Nom du produit")
    category: Literal['Electronics', 'Audio', 'Accessories']
    quantity: int = Field(gt=0, description="Quantite commandee")
    unit_price: float = Field(gt=0, description="Prix unitaire")
    customer_id: str = Field(pattern=r'^C\d{3}$', description="ID client (format Cxxx)")
    city: str = Field(min_length=1, description="Ville de livraison")
    total: float = Field(default=0, description="Montant total (calcule auto)")
    
    @field_validator('date')
    @classmethod
    def valider_date(cls, v):
        """Verifie que la date n'est pas dans le futur"""
        if v > datetime.now():
            raise ValueError("La date ne peut pas etre dans le futur")
        return v
    
    @field_validator('product')
    @classmethod
    def valider_product(cls, v):
        """Nettoie et valide le nom du produit"""
        v = v.strip()
        if not v:
            raise ValueError("Le nom du produit ne peut pas etre vide")
        return v
    
    @model_validator(mode='after')
    def calculer_total(self):
        """Calcule automatiquement le total"""
        self.total = round(self.quantity * self.unit_price, 2)
        return self
    
    class Config:
        json_schema_extra = {
            "example": {
                "order_id": 1,
                "date": "2024-01-15T10:30:00",
                "product": "Laptop",
                "category": "Electronics",
                "quantity": 2,
                "unit_price": 999.99,
                "customer_id": "C001",
                "city": "Paris"
            }
        }


print("=== VALIDATION AVEC PYDANTIC ===")

# Test 1 : Commande valide
print("\n1. Commande valide :")
try:
    order1 = Order(
        order_id=1,
        date=datetime(2024, 1, 15, 10, 30),
        product="Laptop",
        category="Electronics",
        quantity=2,
        unit_price=999.99,
        customer_id="C001",
        city="Paris"
    )
    print(f"OK : {order1}")
    print(f"Total calcule : {order1.total} EUR")
except Exception as e:
    print(f"Erreur : {e}")

# Test 2 : order_id negatif
print("\n2. order_id negatif (erreur attendue) :")
try:
    order2 = Order(
        order_id=-1,
        date=datetime(2024, 1, 15),
        product="Phone",
        category="Electronics",
        quantity=1,
        unit_price=699.99,
        customer_id="C002",
        city="Lyon"
    )
except Exception as e:
    print(f"Erreur attrapee : {e}")

# Test 3 : Quantite nulle
print("\n3. Quantite nulle (erreur attendue) :")
try:
    order3 = Order(
        order_id=3,
        date=datetime(2024, 1, 15),
        product="Tablet",
        category="Electronics",
        quantity=0,
        unit_price=449.99,
        customer_id="C003",
        city="Marseille"
    )
except Exception as e:
    print(f"Erreur attrapee : {e}")

# Test 4 : customer_id invalide
print("\n4. customer_id invalide (erreur attendue) :")
try:
    order4 = Order(
        order_id=4,
        date=datetime(2024, 1, 15),
        product="Headphones",
        category="Audio",
        quantity=1,
        unit_price=79.99,
        customer_id="INVALID",  # Format invalide
        city="Toulouse"
    )
except Exception as e:
    print(f"Erreur attrapee : {e}")

# Test 5 : Date future
print("\n5. Date dans le futur (erreur attendue) :")
try:
    order5 = Order(
        order_id=5,
        date=datetime(2030, 1, 1),
        product="Monitor",
        category="Electronics",
        quantity=1,
        unit_price=349.99,
        customer_id="C005",
        city="Bordeaux"
    )
except Exception as e:
    print(f"Erreur attrapee : {e}")

# Test 6 : Categorie invalide
print("\n6. Categorie invalide (erreur attendue) :")
try:
    order6 = Order(
        order_id=6,
        date=datetime(2024, 1, 15),
        product="Book",
        category="Books",  # Pas dans les categories autorisees
        quantity=1,
        unit_price=19.99,
        customer_id="C006",
        city="Nice"
    )
except Exception as e:
    print(f"Erreur attrapee : {e}")

# Test 7 : Validation d'un batch
print("\n7. Validation d'un batch de commandes :")
commandes_test = [
    {"order_id": 10, "date": datetime(2024, 1, 20), "product": "Laptop", "category": "Electronics",
     "quantity": 1, "unit_price": 999.99, "customer_id": "C010", "city": "Paris"},
    {"order_id": 11, "date": datetime(2024, 1, 21), "product": "Phone", "category": "Electronics",
     "quantity": 2, "unit_price": 699.99, "customer_id": "C011", "city": "Lyon"},
    {"order_id": -1, "date": datetime(2024, 1, 22), "product": "Invalid", "category": "Electronics",
     "quantity": 1, "unit_price": 100, "customer_id": "C012", "city": "Nice"},  # Invalide
]

valides = []
invalides = []

for cmd_data in commandes_test:
    try:
        order = Order(**cmd_data)
        valides.append(order)
    except Exception as e:
        invalides.append((cmd_data['order_id'], str(e)))

print(f"Commandes valides : {len(valides)}")
print(f"Commandes invalides : {len(invalides)}")
if invalides:
    print("\nErreurs :")
    for order_id, error in invalides:
        print(f"  Order {order_id} : {error}")

---
## Exercice 4 : Pipeline Complet (DIFFICILE)

Creez un pipeline ETL complet :

**Extract :**
1. Charger le CSV dans Pandas

**Transform :**
2. Nettoyer les donnees :
   - Supprimer les valeurs manquantes
   - Corriger les valeurs aberrantes (quantity < 0, price <= 0)
   - Normaliser les villes (capitalize)
3. Valider chaque ligne avec Pydantic
4. Enrichir avec des colonnes calculees :
   - `month` : mois de la commande
   - `revenue_category` : "Low" (<500), "Medium" (500-1000), "High" (>1000)
   - `is_weekend` : bool (samedi/dimanche)

**Load :**
5. Exporter en Parquet avec compression
6. Sauvegarder un rapport JSON avec les statistiques

Creez une fonction `pipeline_etl(input_file, output_dir)` qui execute tout le pipeline.

In [ ]:
# Votre code ici


### Solution 4

In [ ]:
# Solution
import pandas as pd
import json
from pathlib import Path
from datetime import datetime

def pipeline_etl(df_input, output_dir='output'):
    """
    Pipeline ETL complet pour les donnees de commandes
    """
    print("=== PIPELINE ETL ===")
    print(f"Input : {len(df_input)} lignes\n")
    
    # Creer le dossier de sortie
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    rapport = {
        'timestamp': datetime.now().isoformat(),
        'input_rows': len(df_input),
        'etapes': []
    }
    
    # ========== EXTRACT ==========
    print("[1/6] EXTRACT : Chargement des donnees")
    df = df_input.copy()
    rapport['etapes'].append({'etape': 'extract', 'lignes': len(df)})
    
    # ========== TRANSFORM - Nettoyage ==========
    print("[2/6] TRANSFORM : Nettoyage des donnees")
    
    # Supprimer les valeurs manquantes
    lignes_avant = len(df)
    df = df.dropna()
    lignes_supprimees_na = lignes_avant - len(df)
    print(f"  - Valeurs manquantes : {lignes_supprimees_na} lignes supprimees")
    
    # Corriger les valeurs aberrantes
    lignes_avant = len(df)
    df = df[(df['quantity'] > 0) & (df['unit_price'] > 0)]
    lignes_supprimees_aberrantes = lignes_avant - len(df)
    print(f"  - Valeurs aberrantes : {lignes_supprimees_aberrantes} lignes supprimees")
    
    # Normaliser les villes
    df['city'] = df['city'].str.strip().str.capitalize()
    print(f"  - Villes normalisees")
    
    # Recalculer le total
    df['total'] = df['quantity'] * df['unit_price']
    
    rapport['etapes'].append({
        'etape': 'nettoyage',
        'lignes_supprimees_na': lignes_supprimees_na,
        'lignes_supprimees_aberrantes': lignes_supprimees_aberrantes,
        'lignes_restantes': len(df)
    })
    
    # ========== TRANSFORM - Validation Pydantic ==========
    print("[3/6] TRANSFORM : Validation Pydantic")
    
    lignes_valides = []
    lignes_invalides = []
    
    for idx, row in df.iterrows():
        try:
            # Convertir la ligne en dict et valider
            order_data = {
                'order_id': int(row['order_id']),
                'date': row['date'],
                'product': row['product'],
                'category': row['category'],
                'quantity': int(row['quantity']),
                'unit_price': float(row['unit_price']),
                'customer_id': row['customer_id'],
                'city': row['city']
            }
            order = Order(**order_data)
            lignes_valides.append(idx)
        except Exception as e:
            lignes_invalides.append((idx, str(e)))
    
    print(f"  - Lignes valides : {len(lignes_valides)}")
    print(f"  - Lignes invalides : {len(lignes_invalides)}")
    
    # Garder seulement les lignes valides
    df = df.loc[lignes_valides]
    
    rapport['etapes'].append({
        'etape': 'validation',
        'lignes_valides': len(lignes_valides),
        'lignes_invalides': len(lignes_invalides)
    })
    
    # ========== TRANSFORM - Enrichissement ==========
    print("[4/6] TRANSFORM : Enrichissement")
    
    # Extraire le mois
    df['month'] = df['date'].dt.to_period('M').astype(str)
    
    # Categorie de revenu
    def categoriser_revenu(total):
        if total < 500:
            return 'Low'
        elif total <= 1000:
            return 'Medium'
        else:
            return 'High'
    
    df['revenue_category'] = df['total'].apply(categoriser_revenu)
    
    # Weekend
    df['is_weekend'] = df['date'].dt.dayofweek.isin([5, 6])
    
    print(f"  - Colonnes ajoutees : month, revenue_category, is_weekend")
    
    rapport['etapes'].append({
        'etape': 'enrichissement',
        'colonnes_ajoutees': ['month', 'revenue_category', 'is_weekend']
    })
    
    # ========== LOAD - Export Parquet ==========
    print("[5/6] LOAD : Export Parquet")
    
    parquet_path = output_path / 'orders_clean.parquet'
    df.to_parquet(parquet_path, compression='snappy', index=False)
    
    file_size = parquet_path.stat().st_size / 1024  # KB
    print(f"  - Fichier : {parquet_path}")
    print(f"  - Taille : {file_size:.2f} KB")
    
    rapport['output_file'] = str(parquet_path)
    rapport['file_size_kb'] = round(file_size, 2)
    
    # ========== LOAD - Rapport ==========
    print("[6/6] LOAD : Generation du rapport")
    
    # Statistiques finales
    statistiques = {
        'nb_commandes': len(df),
        'ca_total': float(df['total'].sum()),
        'panier_moyen': float(df['total'].mean()),
        'nb_clients': df['customer_id'].nunique(),
        'nb_produits': df['product'].nunique(),
        'nb_villes': df['city'].nunique(),
        'periode': {
            'debut': df['date'].min().isoformat(),
            'fin': df['date'].max().isoformat()
        },
        'repartition_revenue_category': df['revenue_category'].value_counts().to_dict(),
        'commandes_weekend': int(df['is_weekend'].sum()),
        'top_3_produits': df.groupby('product')['total'].sum().nlargest(3).to_dict()
    }
    
    rapport['statistiques'] = statistiques
    rapport['output_rows'] = len(df)
    
    # Sauvegarder le rapport
    rapport_path = output_path / 'pipeline_report.json'
    with open(rapport_path, 'w', encoding='utf-8') as f:
        json.dump(rapport, f, indent=2, ensure_ascii=False)
    
    print(f"  - Rapport : {rapport_path}")
    
    # ========== Résumé ==========
    print("\n=== RESUME ===")
    print(f"Lignes input : {rapport['input_rows']}")
    print(f"Lignes output : {rapport['output_rows']}")
    print(f"Taux de succes : {rapport['output_rows']/rapport['input_rows']*100:.1f}%")
    print(f"\nCA total : {statistiques['ca_total']:,.2f} EUR")
    print(f"Panier moyen : {statistiques['panier_moyen']:.2f} EUR")
    print(f"Clients uniques : {statistiques['nb_clients']}")
    
    return df, rapport


# Executer le pipeline
df_clean, rapport = pipeline_etl(df)

# Afficher un apercu
print("\n=== APERCU DES DONNEES NETTOYEES ===")
print(df_clean.head(10))

print("\n=== NOUVELLES COLONNES ===")
print(df_clean[['order_id', 'date', 'total', 'month', 'revenue_category', 'is_weekend']].head())

---
## Exercice 5 : Analyse avec DuckDB (DIFFICILE)

Utilisez DuckDB pour des requetes SQL sur le fichier Parquet :

**Requetes a implementer :**

1. **Top 5 des produits** par CA total
2. **CA mensuel** avec croissance MoM (Month over Month)
3. **Analyse par ville** : CA, nombre de commandes, panier moyen
4. **Cohort analysis** : clients par mois de premiere commande
5. **Window functions** : Rang des produits par ville selon le CA
6. **Analyse weekend vs semaine** : comparaison des performances

Utilisez des requetes SQL avancees avec window functions, CTEs, et agregations.

In [ ]:
# Votre code ici


### Solution 5

In [ ]:
# Solution
import duckdb

print("=== ANALYSE AVEC DUCKDB ===")

# Connexion DuckDB (en memoire)
conn = duckdb.connect()

# Charger le Parquet
parquet_file = 'output/orders_clean.parquet'
print(f"\nChargement : {parquet_file}\n")

# ========== Requete 1 : Top 5 produits ==========
print("1. TOP 5 DES PRODUITS PAR CA")
print("=" * 60)

query1 = f"""
SELECT 
    product,
    COUNT(*) as nb_commandes,
    SUM(quantity) as quantite_totale,
    ROUND(SUM(total), 2) as ca_total,
    ROUND(AVG(total), 2) as panier_moyen
FROM '{parquet_file}'
GROUP BY product
ORDER BY ca_total DESC
LIMIT 5
"""

result1 = conn.execute(query1).df()
print(result1.to_string(index=False))

# ========== Requete 2 : CA mensuel avec croissance ==========
print("\n\n2. CA MENSUEL AVEC CROISSANCE MoM")
print("=" * 60)

query2 = f"""
WITH monthly_revenue AS (
    SELECT 
        month,
        COUNT(*) as nb_commandes,
        ROUND(SUM(total), 2) as ca_total
    FROM '{parquet_file}'
    GROUP BY month
    ORDER BY month
)
SELECT 
    month,
    nb_commandes,
    ca_total,
    LAG(ca_total) OVER (ORDER BY month) as ca_mois_precedent,
    ROUND(
        (ca_total - LAG(ca_total) OVER (ORDER BY month)) / 
        LAG(ca_total) OVER (ORDER BY month) * 100, 
        2
    ) as croissance_pct
FROM monthly_revenue
"""

result2 = conn.execute(query2).df()
print(result2.to_string(index=False))

# ========== Requete 3 : Analyse par ville ==========
print("\n\n3. ANALYSE PAR VILLE")
print("=" * 60)

query3 = f"""
SELECT 
    city,
    COUNT(*) as nb_commandes,
    COUNT(DISTINCT customer_id) as nb_clients,
    ROUND(SUM(total), 2) as ca_total,
    ROUND(AVG(total), 2) as panier_moyen,
    ROUND(SUM(total) * 100.0 / SUM(SUM(total)) OVER (), 2) as pct_ca_total
FROM '{parquet_file}'
GROUP BY city
ORDER BY ca_total DESC
"""

result3 = conn.execute(query3).df()
print(result3.to_string(index=False))

# ========== Requete 4 : Cohort analysis ==========
print("\n\n4. COHORT ANALYSIS : Premiere commande par client")
print("=" * 60)

query4 = f"""
WITH first_order AS (
    SELECT 
        customer_id,
        MIN(month) as cohort_month,
        MIN(date) as first_order_date
    FROM '{parquet_file}'
    GROUP BY customer_id
)
SELECT 
    cohort_month,
    COUNT(DISTINCT customer_id) as nb_nouveaux_clients,
    ROUND(AVG(CAST(strftime(first_order_date, '%d') AS INTEGER)), 1) as jour_moyen
FROM first_order
GROUP BY cohort_month
ORDER BY cohort_month
"""

result4 = conn.execute(query4).df()
print(result4.to_string(index=False))

# ========== Requete 5 : Rang des produits par ville ==========
print("\n\n5. RANG DES PRODUITS PAR VILLE (TOP 2 PAR VILLE)")
print("=" * 60)

query5 = f"""
WITH product_city_revenue AS (
    SELECT 
        city,
        product,
        ROUND(SUM(total), 2) as ca_total,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY SUM(total) DESC) as rang
    FROM '{parquet_file}'
    GROUP BY city, product
)
SELECT 
    city,
    rang,
    product,
    ca_total
FROM product_city_revenue
WHERE rang <= 2
ORDER BY city, rang
"""

result5 = conn.execute(query5).df()
print(result5.to_string(index=False))

# ========== Requete 6 : Weekend vs Semaine ==========
print("\n\n6. ANALYSE WEEKEND VS SEMAINE")
print("=" * 60)

query6 = f"""
SELECT 
    CASE 
        WHEN is_weekend THEN 'Weekend'
        ELSE 'Semaine'
    END as periode,
    COUNT(*) as nb_commandes,
    ROUND(SUM(total), 2) as ca_total,
    ROUND(AVG(total), 2) as panier_moyen,
    ROUND(AVG(quantity), 2) as quantite_moyenne,
    ROUND(SUM(total) * 100.0 / SUM(SUM(total)) OVER (), 2) as pct_ca
FROM '{parquet_file}'
GROUP BY is_weekend
ORDER BY periode DESC
"""

result6 = conn.execute(query6).df()
print(result6.to_string(index=False))

# ========== Requete BONUS : Analyse de revenue_category ==========
print("\n\nBONUS : ANALYSE PAR CATEGORIE DE REVENU")
print("=" * 60)

query_bonus = f"""
SELECT 
    revenue_category,
    COUNT(*) as nb_commandes,
    ROUND(SUM(total), 2) as ca_total,
    ROUND(MIN(total), 2) as montant_min,
    ROUND(MAX(total), 2) as montant_max,
    ROUND(AVG(total), 2) as montant_moyen,
    ROUND(SUM(total) * 100.0 / SUM(SUM(total)) OVER (), 2) as pct_ca
FROM '{parquet_file}'
GROUP BY revenue_category
ORDER BY 
    CASE revenue_category 
        WHEN 'Low' THEN 1 
        WHEN 'Medium' THEN 2 
        WHEN 'High' THEN 3 
    END
"""

result_bonus = conn.execute(query_bonus).df()
print(result_bonus.to_string(index=False))

# Fermer la connexion
conn.close()

print("\n\n=== ANALYSE TERMINEE ===")

---
## Conclusion

Felicitations ! Vous avez complete un projet complet de data engineering.

### Competences acquises :

**Pandas :**
- Exploration de donnees (info, describe, value_counts)
- Manipulation et transformation (groupby, pivot_table)
- Agregations avancees

**Pydantic :**
- Validation de schemas de donnees
- Validateurs personnalises
- Gestion d'erreurs de validation

**Pipeline ETL :**
- Extract : chargement de donnees
- Transform : nettoyage, validation, enrichissement
- Load : export Parquet, generation de rapports

**DuckDB :**
- Requetes SQL sur fichiers Parquet
- Window functions (ROW_NUMBER, LAG)
- CTEs (Common Table Expressions)
- Agregations et analyses complexes

### Pour aller plus loin :
- Implementer un systeme de logging
- Ajouter des tests unitaires
- Creer des visualisations avec matplotlib/plotly
- Scheduler le pipeline avec Apache Airflow
- Deployer sur le cloud (AWS S3, GCP BigQuery)